In [ ]:
import pandas as pd
import numpy as np

import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir('C:/Users/aecat/Dropbox/Docencia/PUC/Magíster Ciencia de Datos/Datos')

## Carga de información

In [3]:
laliga2122 = pd.read_excel('../Semana 3/datos_liga/LaLiga21 22.xlsx', )
laliga2021 = pd.read_excel('../Semana 3/datos_liga/LaLiga20 21.xlsx', )
laliga1920 = pd.read_excel('../Semana 3/datos_liga/LaLiga19 20.xlsx', )
laliga1819 = pd.read_excel('../Semana 3/datos_liga/LaLiga18 19.xlsx', )
laliga1718 = pd.read_excel('../Semana 3/datos_liga/LaLiga17 18.xlsx', )
laliga1617 = pd.read_excel('../Semana 3/datos_liga/LaLiga16 17.xlsx', )
seg2122 =  pd.read_excel('../Semana 3/datos_liga/LaLiga123 21 22.xlsx', )
seg2021 =  pd.read_excel('../Semana 3/datos_liga/LaLiga123 20 21.xlsx', )
seg1920 =  pd.read_excel('../Semana 3/datos_liga/LaLiga123 19 20.xlsx', )
seg1819 =  pd.read_excel('../Semana 3/datos_liga/LaLiga123 18 19.xlsx', )
seg1718 =  pd.read_excel('../Semana 3/datos_liga/LaLiga123 17 18.xlsx', )
seg1617 =  pd.read_excel('../Semana 3/datos_liga/LaLiga123 16 17.xlsx', )

In [4]:
laliga = laliga2122.dropna(how='all').append(laliga2021.dropna(how='all')
).append(laliga1920.dropna(how='all')).append(laliga1819.dropna(how='all')
).append(laliga1718.dropna(how='all')).append(laliga1617.dropna(how='all')).loc[:,
['Wk','Date','Local','xG','Score','xG.1','Visitante', 'Attendance']]
laliga['Attendance']= laliga['Attendance']*1000

In [5]:
def total_goles(string):
    goles = string.split('–')
    total_goles = int(goles[0])+int(goles[1])
    return total_goles

laliga['Total de goles']= laliga['Score'].apply(total_goles)

In [6]:
def torneo(año,mes):
    if ((año==2016) & (mes<=12)|(año==2017) & (mes<8)):
        return '16/17'
    elif ((año==2017) & (mes<=12)|(año==2018) & (mes<8)):
        return '17/18'
    elif ((año==2018) & (mes<=12)|(año==2019) & (mes<8)):
        return '18/19'
    elif ((año==2019) & (mes<=12)|(año==2020) & (mes<9)):
        return '19/20'
    elif ((año==2020) & (mes<=12)|(año==2021) & (mes<8)):
        return '20/21'
    elif ((año==2021) & (mes<=12)|(año==2022) & (mes<8)):
        return '21/22'

    
laliga['Año'] = laliga['Date'].dt.year
laliga['Mes'] = laliga['Date'].dt.month
laliga['Temporada'] = laliga.apply(lambda x: torneo(x['Año'],x['Mes']),axis = 1)

## Análisis de victorias

Crearemos columnas para analizar los resultados de los partidos de cada torneo. Primero, una columna que nos diga si ganó el equipo local o visitante y otra columna con el nombre del equipo que ganó

In [7]:
#esta función nos indicará si el partido lo ganó el Local, el Visitante o si fue empate
def resultado(string):
    goles = string.split('–')
    if int(goles[0])>int(goles[1]):
        return 'Local'
    if int(goles[0])<int(goles[1]):
        return 'Visitante'
    else: 
        return 'Empate'

#esta función nos indicará si el partido lo ganó el Local y nos devolverá el nombre del equipo local, 
#el Visitante y nos devolverá el nombre del equipo visitante o si fue empate, y nos devolverá el string "Empate"
def resultado_equipo(string, local, visita):
    goles = string.split('–')
    if int(goles[0])>int(goles[1]):
        return local
    if int(goles[0])<int(goles[1]):
        return visita
    else: 
        return 'Empate'
    
laliga['Resultado'] = laliga['Score'].apply(resultado)
laliga['Ganador'] = laliga.apply(lambda x: resultado_equipo(x['Score'],x['Local'],x['Visitante']),axis=1)

Ahora, podemos estudiar la cantidad de partidos que ganó el equipo local, visita o fue empate por cada torneo.


In [8]:
condicion_predominante = laliga.groupby(['Temporada','Resultado'])['Wk'].count().reset_index() #agrupamos la información del Dataframe laliga por Temporada y Resultado, y contamos la cantidad de partidos que caen en cada agrupación.
condicion_predominante = condicion_predominante.set_axis(
['Temporada', 'Resultado','Cantidad'], axis='columns') #colocamos nombres a las columnas del nuevo Dataframe

condicion_predominante

,Temporada,Resultado,Cantidad
0,16/17,Empate,89
1,16/17,Local,181
2,16/17,Visitante,110
3,17/18,Empate,86
4,17/18,Local,179
5,17/18,Visitante,115
6,18/19,Empate,110
7,18/19,Local,168
8,18/19,Visitante,102
9,19/20,Empate,105


Si solo nos interesa el histórico, sin considerar la diferencia por temporada, tenemos que :

In [9]:
cond_predominante_abs = laliga.groupby(['Resultado'])['Wk'].count().reset_index() #ahora solamente agrupamos por resultado
cond_predominante_abs = cond_predominante_abs.set_axis(
['Resultado','Cantidad'], axis='columns')

cond_predominante_abs

,Resultado,Cantidad
0,Empate,610
1,Local,1025
2,Visitante,645


De donde se puede notar una clara tendencia a la victoria del equipo local.

## Más ganadores
Ahora, queremos saber cuáles son los equipos que más han ganado durante las últimas 6 temporadas de la primera división española.

In [10]:
mas_ganadores = laliga[laliga['Ganador']!= 'Empate'].groupby(['Ganador'])['Wk'].count().reset_index()
mas_ganadores = mas_ganadores.set_axis(['Equipo','Cantidad de victorias'], axis='columns')
mas_ganadores = mas_ganadores.sort_values(by= 'Cantidad de victorias', ascending = False)
mas_ganadores.head(3)

,Equipo,Cantidad de victorias
3,Barcelona,152
22,Real Madrid,149
2,Atlético Madrid,133


## Cantidad de partidos con x goles por torneo 

También, nos puede interesar la cantidad de goles que se han realizado en un torneo:

In [11]:
#construimos esta función, a la que le entregamos un marcador (Score) y nos retorna si la cantidad total de goles de ese
#marcador es 0, 1, 2, 3 o más de 4.
def intervalos(score):
    goles = score.split('–')
    total_goles = int(goles[0])+int(goles[1])
    if total_goles <1:
        return '0'
    elif total_goles<2:
        return '1'
    elif total_goles<3:
        return '2'
    elif total_goles<4:
        return '3'
    elif total_goles>=4:
        return '4+'
    
laliga['Clasificación goles'] = laliga['Score'].apply(intervalos)

Queremos saber la cantidad de partidos, por temporada, en que se han convertido 0, 1, 2, 3 o más de 4 goles.

In [12]:
cant_goles_pp = laliga.groupby(['Temporada','Clasificación goles'])['Date'].count().reset_index() #agrupamos la información del Dataframe laliga por temporada y clasificación de goles, y contamos cuantos partidos hay en cada agrupación
cant_goles_pp = cant_goles_pp.set_axis(['Temporada','Cantidad de goles por partido','Cantidad de partidos'],
                axis= 'columns') #nombramos las columnas
cant_goles_pp.head(10)

,Temporada,Cantidad de goles por partido,Cantidad de partidos
0,16/17,0,27
1,16/17,1,57
2,16/17,2,82
3,16/17,3,86
4,16/17,4+,128
5,17/18,0,28
6,17/18,1,82
7,17/18,2,78
8,17/18,3,84
9,17/18,4+,108


## Ejercicio formativo 1
¿Cuál de las siguientes alternativas corresponde al código que permite agrupar los resultados de localía por equipo?

In [13]:
local_result = laliga.groupby(['Temporada','Local','Visitante','Score'])['Wk'].count().reset_index()
local_result = local_result.drop('Wk',axis=1)
local_result

,Temporada,Local,Visitante,Score
0,16/17,Alavés,Athletic Club,1–0
1,16/17,Alavés,Atlético Madrid,0–0
2,16/17,Alavés,Barcelona,0–6
3,16/17,Alavés,Betis,1–0
4,16/17,Alavés,Celta Vigo,3–1
...,...,...,...,...
2275,21/22,Villarreal,Rayo Vallecano,2–0
2276,21/22,Villarreal,Real Madrid,0–0
2277,21/22,Villarreal,Real Sociedad,1–2
2278,21/22,Villarreal,Sevilla,1–1
